In [3]:
import keras
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.models import Sequential 
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.layers.core import Dense
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.models import Sequential

## load data

In [21]:
X = pd.read_csv('datasets/traindata.csv',header=None)
Y = pd.read_csv('datasets/trainlabel_onehot.csv',header=None)
testdata = pd.read_csv('datasets/testdata.csv',header=None)
testlabel = pd.read_csv('datasets/testlabel_onehot.csv',header=None)

## Data normalization

In [22]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

x = ss.fit_transform(X)
testdata = ss.transform(testdata)

/Users/huayanpei/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/huayanpei/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/huayanpei/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


## CNN

In [34]:
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

In [25]:
x_cnn = np.reshape(x, (x.shape[0],x.shape[1],1))
testdata_cnn = np.reshape(testdata, (testdata.shape[0], testdata.shape[1], 1))

In [82]:
model = Sequential()
model.add(Conv1D(256,28, padding="same", activation='relu', input_shape=(28,1)))
model.add(MaxPooling1D(pool_size=(2)))
#model.add(Conv1D(128, 28, padding="same",activation='relu'))
#model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(5, activation="softmax"))

In [83]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [80]:
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
            TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]

In [84]:
callbacks = [ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
            TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]

In [85]:
model.fit(x_cnn, Y,epochs=10, callbacks=callbacks,validation_data=(testdata_cnn,testlabel))

Train on 125973 samples, validate on 22544 samples
Epoch 1/10
125973/125973 [==============================] - 16s 130us/step - loss: 0.0709 - acc: 0.9778 - val_loss: 1.7029 - val_acc: 0.7411
Epoch 2/10
125973/125973 [==============================] - 16s 127us/step - loss: 0.0446 - acc: 0.9850 - val_loss: 2.1514 - val_acc: 0.7465
Epoch 3/10
125973/125973 [==============================] - 16s 129us/step - loss: 0.0392 - acc: 0.9877 - val_loss: 1.9499 - val_acc: 0.7793
Epoch 4/10
125973/125973 [==============================] - 16s 129us/step - loss: 0.0364 - acc: 0.9887 - val_loss: 2.2416 - val_acc: 0.7561
Epoch 5/10
125973/125973 [==============================] - 16s 128us/step - loss: 0.0331 - acc: 0.9892 - val_loss: 2.1483 - val_acc: 0.7762
Epoch 6/10
125973/125973 [==============================] - 16s 130us/step - loss: 0.0320 - acc: 0.9902 - val_loss: 2.0528 - val_acc: 0.7709
Epoch 7/10
125973/125973 [==============================] - 16s 131us/step - loss: 0.0301 - acc: 0.9905

In [86]:
X_predict = model.predict(testdata_cnn)
for i in range(len(X_predict)):
    max_value = max(X_predict[i])
    for j in range(len(X_predict[i])):
        if max_value == X_predict[i][j]:
            X_predict[i][j] = 1
        else:
            X_predict[i][j] = 0

In [87]:
from sklearn.metrics import classification_report
print(classification_report(X_predict, testlabel))

              precision    recall  f1-score   support

           0       0.97      0.70      0.81     13482
           1       0.00      0.00      0.00         0
           2       0.18      0.94      0.31       533
           3       0.72      0.82      0.76      2139
           4       0.81      0.95      0.88      6390

   micro avg       0.79      0.79      0.79     22544
   macro avg       0.54      0.68      0.55     22544
weighted avg       0.89      0.79      0.82     22544
 samples avg       0.79      0.79      0.79     22544



### Use Darpa Dataset as validation dataset

#### Import every week's darpa data

In [89]:
Week1 = pd.read_csv('datasets/darpa_5/week1.csv',header=None)
Week1_label = pd.read_csv('datasets/darpa_5/week1_label.csv',header=None)
Week2 = pd.read_csv('datasets/darpa_5/week2.csv',header=None)
Week2_label = pd.read_csv('datasets/darpa_5/week2_label.csv',header=None)
Week3 = pd.read_csv('datasets/darpa_5/week3.csv',header=None)
Week3_label = pd.read_csv('datasets/darpa_5/week3_label.csv',header=None)
Week4 = pd.read_csv('datasets/darpa_5/week4.csv',header=None)
Week4_label = pd.read_csv('datasets/darpa_5/week4_label.csv',header=None)
Week5 = pd.read_csv('datasets/darpa_5/week5.csv',header=None)
Week5_label = pd.read_csv('datasets/darpa_5/week5_label.csv',header=None)
Week6 = pd.read_csv('datasets/darpa_5/week6.csv',header=None)
Week6_label = pd.read_csv('datasets/darpa_5/week6_label.csv',header=None)
Week7 = pd.read_csv('datasets/darpa_5/week7.csv',header=None)
Week7_label = pd.read_csv('datasets/darpa_5/week7_label.csv',header=None)

In [90]:
Xall = pd.concat([Week1,Week2,Week3, Week4, Week5, Week6, Week7], axis=0, ignore_index=True)
Yall = pd.concat([Week1_label,Week2_label,Week3_label, Week4_label, Week5_label, Week6_label, Week7_label], axis=0, ignore_index=True)

X_all = ss.transform(Xall)

/Users/huayanpei/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [93]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(n_values=5)
def onehot(self):
    return encoder.fit_transform(self.values.reshape(-1,1)).toarray()

Yall_oh = onehot(Yall)

In [95]:
darpadata_cnn = np.reshape(X_all, (X_all.shape[0], X_all.shape[1], 1)) 

### CNN use Darpa dataset , accuracy 92%

In [96]:
cnndarpa = model.predict(darpadata_cnn)

In [99]:
for i in range(len(cnndarpa)):
    max_value = max(cnndarpa[i])
    for j in range(len(cnndarpa[i])):
        if max_value == cnndarpa[i][j]:
            cnndarpa[i][j] = 1
        else:
            cnndarpa[i][j] = 0

In [102]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
print(accuracy_score(cnndarpa, Yall_oh))
print(classification_report(cnndarpa, Yall_oh))

0.915772050698108


/Users/huayanpei/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       0.93      0.90      0.92    920052
           1       0.00      0.00      0.00         0
           2       0.11      0.00      0.00      9636
           3       0.17      0.12      0.14     68273
           4       0.93      0.98      0.96   1083379

   micro avg       0.92      0.92      0.92   2081340
   macro avg       0.43      0.40      0.40   2081340
weighted avg       0.90      0.92      0.91   2081340
 samples avg       0.92      0.92      0.92   2081340



## SVM